In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Koneksi ke database OLTP dan Data Warehouse
oltp_engine = create_engine("jdbc:postgresql://localhost:5432/oltp")
dw_engine = create_engine("jdbc:postgresql://localhost:5432/warehouse")


In [ ]:
# Extract data dari OLTP
users_df = pd.read_sql('SELECT * FROM tb_users', oltp_engine)
shippers_df = pd.read_sql('SELECT * FROM tb_shippers', oltp_engine)
payments_df = pd.read_sql('SELECT * FROM tb_payments', oltp_engine)
ratings_df = pd.read_sql('SELECT * FROM tb_ratings', oltp_engine)
vouchers_df = pd.read_sql('SELECT * FROM tb_vouchers', oltp_engine)
orders_df = pd.read_sql('SELECT * FROM tb_orders', oltp_engine)
products_df = pd.read_sql('SELECT * FROM tb_products', oltp_engine)
order_items_df = pd.read_sql('SELECT * FROM tb_order_items', oltp_engine)
product_category_df = pd.read_sql('SELECT * FROM tb_product_category', oltp_engine)

In [ ]:
order_df = order_df.merge(users_df, on='user_id')
order_df = order_df.merge(payments_df, on='payment_id')
order_df = order_df.merge(shippers_df, on='shipper_id')
order_df = order_df.merge(ratings_df, on='rating_id')
order_df = order_df.merge(vouchers_df, how='left', on='voucher_id')

products_df = products_df.merge(product_category_df,how='left', on='product_category_id')
order_items_df = order_items_df.merge(products_df,how='left', on='product_id')


In [ ]:
# Load Dim User
users_df.to_sql('dim_user', dw_engine, if_exists='append', index=False)

# Load Dim Shipper
shippers_df.to_sql('dim_shipper', dw_engine, if_exists='append', index=False)

# Load Dim Payment
payments_df.to_sql('dim_payment', dw_engine, if_exists='append', index=False)

# Load Dim Rating
ratings_df.to_sql('dim_rating', dw_engine, if_exists='append', index=False)

# Load Dim Voucher
vouchers_df.to_sql('dim_voucher', dw_engine, if_exists='append', index=False)

# Load Fact Orders
orders_df.to_sql('fact_orders', dw_engine, if_exists='append', index=False)

# Load Dim Product Category
product_category_df.to_sql('dim_product_category', dw_engine, if_exists='append', index=False)

# Load Dim Product
products_df.to_sql('dim_product', dw_engine, if_exists='append', index=False)

# Load Fact Order Item
order_items_df.to_sql('fact_order_item', dw_engine, if_exists='append', index=False)

print("ETL process completed successfully.")